**REF:** The following implementation is adapted and inspired by
https://keras.io/examples/structured_data/collaborative_filtering_movielens/
Further details are provided in the associated report.


In [ ]:
#Imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd

## **Processing Final Dataset**

In [ ]:
# Defining Functions
def preprocess(rating_df):
  user_encode = {x: i for i, x in enumerate(rating_df["user_id"].unique().tolist())}
  book_ids = rating_df["book_id"].unique().tolist()
  book_encode = {x: i for i, x in enumerate(book_ids)}
  t_book_encode = {i: x for i, x in enumerate(book_ids)}
  rating_df["user"] = rating_df["user_id"].map(user_encode)
  rating_df["book"] = rating_df["book_id"].map(book_encode)
  rating_df["rating"] = rating_df["rating"].values.astype(np.float32)
  min_rating = min(rating_df["rating"])
  max_rating = max(rating_df["rating"])
  return rating_df, user_encode, book_encode, t_book_encode, min_rating, max_rating

In [ ]:
rating_df = pd.read_csv("goodreads_ratings.csv")
rating_df, user_encode, book_encode, t_book_encode, min_rating, max_rating = preprocess(df)

num_users = len(user_encode)
num_books = len(t_book_encode)
num_ratings = len(rating_df)

#This code segment was taken from https://stackoverflow.com/questions/38708621/how-to-calculate-percentage-of-sparsity-for-a-numpy-array-matrix
user_item_df = rating_df.pivot(index='user_id', columns='book_id',  values='rating').fillna(0)
non_zero = np.count_nonzero(user_item_df)
total_val = np.product(user_item_df.shape)
sparsity = ((total_val - non_zero) / total_val)*100

print(f"# Users: {num_users}")
print(f"# Movies: {num_books}")
print(f"# Ratings: {num_ratings}")
print(f"Data sparsity: {sparsity}")

# Users: 228
# Movies: 59139
# Ratings: 100000
Data sparsity: 99.67277508267024


## **Model Training & Evaluation**

In [ ]:
rating_df = rating_df.sample(frac=1, random_state=42)
x = rating_df[["user", "book"]].values

# Normalize rating targets between 0 and 1
y = rating_df["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
# Training on 90% of the data and validating on 10%.
train_indices = int(0.9 * rating_df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

In [ ]:
class NCF(keras.Model):
    def __init__(self, num_users, num_books, embedding_size, **kwargs):
        super().__init__(**kwargs)

        # Define all the layers of the model in the constructor
        self.user_embedding = layers.Embedding(num_users, embedding_size, embeddings_initializer="he_normal", embeddings_regularizer=keras.regularizers.l2(1e-6))
        self.user_bias = layers.Embedding(num_users, 1)
        self.book_embedding = layers.Embedding(num_books, embedding_size, embeddings_initializer="he_normal", embeddings_regularizer=keras.regularizers.l2(1e-6))
        self.book_bias = layers.Embedding(num_books, 1)
        self.dot = layers.Dot(axes=-1)
        self.sigmoid = layers.Activation('sigmoid')

    def call(self, inputs):
        # Use the defined layers in the call method
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        book_vector = self.book_embedding(inputs[:, 1])
        book_bias = self.book_bias(inputs[:, 1])

        dot_user_book = self.dot([user_vector, book_vector])
        x = layers.add([dot_user_book, user_bias, book_bias])
        return self.sigmoid(x)

In [ ]:
# Define the model
EMBEDDING_SIZE = 50
model = NCF(num_users, num_books, EMBEDDING_SIZE)

# Use this segment for RMSE Evaluation
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Use this segment for Precision and Recall Evaluation
#model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Train the model based on data split
model.fit(x_train, y_train, batch_size=64, epochs=40, verbose=1, validation_data=(x_val, y_val),)

Epoch 1/40
1407/1407 [==============================] - 36s 25ms/step - loss: 0.6323 - root_mean_squared_error: 0.4018 - val_loss: 0.5904 - val_root_mean_squared_error: 0.3772
Epoch 2/40
1407/1407 [==============================] - 35s 25ms/step - loss: 0.4929 - root_mean_squared_error: 0.3149 - val_loss: 0.5619 - val_root_mean_squared_error: 0.3585
Epoch 3/40
1407/1407 [==============================] - 38s 27ms/step - loss: 0.3279 - root_mean_squared_error: 0.1897 - val_loss: 0.5661 - val_root_mean_squared_error: 0.3570
Epoch 4/40
1407/1407 [==============================] - 40s 28ms/step - loss: 0.2623 - root_mean_squared_error: 0.1241 - val_loss: 0.5732 - val_root_mean_squared_error: 0.3580
Epoch 5/40
1407/1407 [==============================] - 41s 29ms/step - loss: 0.2408 - root_mean_squared_error: 0.0981 - val_loss: 0.5800 - val_root_mean_squared_error: 0.3593
Epoch 6/40
1407/1407 [==============================] - 38s 27ms/step - loss: 0.2314 - root_mean_squared_error: 0.0840 -

In [ ]:
# Save the model
model.save("ncf-model")